# Deep Research API Testing Notebook

This notebook guides you through testing the Deep Research workflow using the **Azure AI Agents Service (Azure AI Foundry)** backed API you built. It mirrors the structure of the Agent Creation notebook and focuses on:

1. Ensuring the API container is running locally
2. Verifying environment configuration (including `AGENT_ID` from the previous notebook)
3. Submitting a Deep Research job
4. Polling job status and retrieving formatted results
5. Downloading the research report (Markdown/PDF)

Run cells in order. Make sure you already created an agent and captured its `agent_id` from the Agent Creation notebook.

## 1. Run the API Locally with Uvicorn (No Docker)
The API auto-loads configuration from `api/.env`. Use these CLI commands.

### 1.1 (Optional) Create & Activate a Virtual Environment (PowerShell)
```powershell
python -m venv .venv
.\.venv\Scripts\Activate.ps1
```

### 1.2 Install Dependencies
```powershell
pip install -r api/requirements.txt
```

### 1.3 Start the Server
From the repository root:
```powershell
uvicorn api.main:app --reload --host 0.0.0.0 --port 8000
```

### 1.4 Verify
Open: http://localhost:8000/docs
You should see endpoints such as:
- /thread/create
- /deep_research/submit
- /deep_research/{thread_id}/{run_id}/status
- /deep_research/{thread_id}/{run_id}/download

Press Ctrl+C to stop the server when done.

Proceed when the server is running.

## 2. Set `AGENT_ID` in `.env`
Copy the `agent.id` printed in the Agent Creation notebook and add it to your `.env`:

```
AGENT_ID=agent_xxxxxxxx
```

Confirm your `.env` also includes:
- `PROJECT_ENDPOINT`
- `BING_RESOURCE_NAME`
- `DEEP_RESEARCH_MODEL_DEPLOYMENT_NAME`
- `MODEL_DEPLOYMENT_NAME`

Reload the environment (restart container if needed) after updating the file.

## 3. Load Environment & Define Base URL
This cell loads the `.env` file and sets the API base URL used in subsequent requests.

In [4]:
# 3. Load environment and define base URL
import os
from dotenv import load_dotenv

load_dotenv('../api/.env', override=True)
BASE_URL = os.getenv("API_BASE_URL", "http://localhost:8000")
AGENT_ID = os.getenv("AGENT_ID")

if not AGENT_ID:
    raise ValueError("AGENT_ID not found in environment. Set it in .env before continuing.")

print("Base URL:", BASE_URL)
print("Agent ID:", AGENT_ID)

Base URL: http://localhost:8000
Agent ID: asst_aMft24CsmWMChmkb30ai30tD


## 4. Create a Thread
A thread groups the conversation and research context. We call the `/thread/create` endpoint to obtain a `thread_id`.

In [6]:
# 4. Create thread
import requests

resp = requests.post(f"{BASE_URL}/thread/create")
resp.raise_for_status()
thread_id = resp.json()["thread_id"]
print("Thread ID:", thread_id)

Thread ID: thread_aEiEPYq61wtSiZcmBR6FLWLX


## 5. Submit a Deep Research Job
We send the research query to `/deep_research/submit` including the `thread_id` query parameter. The response returns a `run_id` we will use to poll status.

In [7]:
# 5. Submit deep research job
query = "Write a report on the cultural impact of the movie Jaws and the effect it had on blockbuster filmmaking"
submit_resp = requests.post(
    f"{BASE_URL}/deep_research/submit",
    params={"thread_id": thread_id},
    json={"query": query}
)
submit_resp.raise_for_status()
submission = submit_resp.json()
run_id = submission["run_id"]
print("Run ID:", run_id)
submission

Run ID: run_gjPs8JaBE3JkHdSXoUsj4KbU


{'thread_id': 'thread_aEiEPYq61wtSiZcmBR6FLWLX',
 'run_id': 'run_gjPs8JaBE3JkHdSXoUsj4KbU',
 'status': 'queued',
 'created_at': '2025-09-08T16:48:42.815052'}

## 6. Poll Job Status
Poll `/deep_research/{thread_id}/{run_id}/status` until status is `completed` or an error state. Adds a short delay between polls to avoid spamming the API.

In [8]:
# 6. Poll job status with live message rendering
import time
from IPython.display import display, Markdown, clear_output
import requests

poll_interval = 5  # seconds
max_wait_seconds = 1200
elapsed = 0
status_payload = None

while elapsed < max_wait_seconds:
    status_resp = requests.get(f"{BASE_URL}/deep_research/{thread_id}/{run_id}/status")
    status_resp.raise_for_status()
    status_payload = status_resp.json()
    status = status_payload["status"].lower()

    # Build markdown view of current status + messages
    messages = status_payload.get("messages") or []
    md_lines = [f"### Run Status: `{status}`", ""]

    if messages:
        for idx, m in enumerate(messages, 1):
            role = (m.get("role") or "").upper()
            content = m.get("content") or ""
            md_lines.append(f"**{idx}. {role}**")
            md_lines.append("")
            md_lines.append(content)
            refs = m.get("references") or []
            if refs:
                md_lines.append("")
                md_lines.append("References:")
                for r in refs:
                    # Each reference: citation_number, title, url
                    try:
                        cnum = r.get("citation_number")
                        title = r.get("title") or "Untitled"
                        url = r.get("url") or ""
                        md_lines.append(f"- [{cnum}] [{title}]({url})")
                    except Exception:
                        pass
            md_lines.append("")
    else:
        md_lines.append("_No messages yet; still processing..._")

    clear_output(wait=True)
    display(Markdown("\n".join(md_lines)))

    if status in ("completed", "failed", "error"):
        break

    time.sleep(poll_interval)
    elapsed += poll_interval

if status_payload is None:
    raise RuntimeError("No status response received.")

# Show final structured payload for programmatic inspection
status_payload

### Run Status: `completed` (elapsed 285s)

**1. USER**

Write a report on the cultural impact of the movie Jaws and the effect it had on blockbuster filmmaking

**2. AGENT**

I will analyze the cultural impact of the movie 'Jaws' and its effects on blockbuster filmmaking. Expect insights on societal influences, industry transformation, and detailed references. I'll provide a well-organized report shortly.

Title: Cultural Impact of Jaws and Its Effect on Blockbuster Filmmaking

 Starting deep research...

**3. AGENT**

**Examining cultural shifts**

I'm digging into how 'Jaws' (1975) impacted societal attitudes, consumer behavior, and movie-making strategies. This involves analyzing box office trends and its lasting cultural legacy. [1](https://www.bing.com/search?q=Jaws%201975%20cultural%20impact%20summer%20blockbuster)

References:
- [1] [Bing Search: 'Jaws 1975 cultural impact summer blockbuster'](https://www.bing.com/search?q=Jaws%201975%20cultural%20impact%20summer%20blockbuster)

**4. AGENT**

**Evaluating sources**

I’m evaluating mainstream and academic sources, considering credibility, and checking for updated info around 2025. Noting potential fan site credibility with "thedailyjaws.com."

**5. AGENT**

**Tracking trends**

I’m gathering Jaws' marketing and its impact on Hollywood, summer blockbusters, and pop culture. Also, considering its fear-inducing influence on societal attitudes and beach tourism. [1](https://www.bing.com/search?q=Jaws%201975%20impact%20fear%20of%20sharks%20public%20behavior)

References:
- [1] [Bing Search: 'Jaws 1975 impact fear of sharks public behavior'](https://www.bing.com/search?q=Jaws%201975%20impact%20fear%20of%20sharks%20public%20behavior)

**6. AGENT**

**Evaluating sources**  
I’m assessing various reputable sources like ABC News, CNN, JSTOR Daily, and blogs by Shark Guardian and Shark Stewards to understand the shark fear narrative, considering their credibility and context.

**7. AGENT**

**Impact assessment and cultural significance**

Examining "Jaws" reveals its significant societal impact, notably its cultural integration and negative effects on shark perceptions.

**8. AGENT**

**Analyzing key insights**

I'm noting the lack of specific impact on beach tourism in the ABC piece, except possibly for a picture caption in line 4. [1](https://www.bing.com/search?q=Jaws%20fear%20of%20water%201975%20beach%20attendance%20drop)

References:
- [1] [Bing Search: 'Jaws fear of water 1975 beach attendance drop'](https://www.bing.com/search?q=Jaws%20fear%20of%20water%201975%20beach%20attendance%20drop)

**9. AGENT**

**Piecing together**

I’m gathering info on sources like thedailyjaws, People.com, Tablet Magazine, and Times of India. Considering their credibility and relevance to Jaws' impact.

**10. AGENT**

**Examining historical shift**

Hmm, I’m thinking about how 'Jaws' introduced mainstream summer blockbusters, creating the summer movie season and affecting beach attendance.

**11. AGENT**

**Noting cultural impact**

Tracking societal attitudes, consumer behavior, and movie-making strategies post-Jaws. The concept of wide summer releases and movie tie-ins emerged from its massive success. [1](https://www.bing.com/search?q=Jaws%201975%20tagline%20water%20again)

References:
- [1] [Bing Search: 'Jaws 1975 tagline water again'](https://www.bing.com/search?q=Jaws%201975%20tagline%20water%20again)

**12. AGENT**

**Gathering sources**  
I’m gathering various credible sources to support the analysis of Jaws' influence on blockbuster filmmaking, focusing on cultural impacts, consumer behavior, and box office trends.

**13. AGENT**

**Weighing image options**

Considering removing the gallery of 51 images and embedding official Jaws movie images with appropriate citations. [1](https://www.bing.com/search?q=Jaws%201975%20Universal%20poster%20image%20embed)

References:
- [1] [Bing Search: 'Jaws 1975 Universal poster image embed'](https://www.bing.com/search?q=Jaws%201975%20Universal%20poster%20image%20embed)

**14. AGENT**

**Pinpointing poster colors**  
Confirming the use of black, white, and red.

**15. AGENT**

**Adjusting my approach**

OK, let me see. I’m thinking of using the direct image link or wiki URL for better visibility, as the current setup hides the image.

**16. AGENT**

**Evaluating image retrieval**

I'm checking the HTML source for a direct image link or using Wikipedia's link. Also, considering inspecting the page through an extension to fetch the image.

**17. AGENT**

**Weighing image options**

OK, I’m pulling together ideas on navigating image licenses for clearer embedding. Considering Yahoo or IndieWire might help with context.

**18. AGENT**

**Tracking Jaws' legacy**

I’m analyzing "Jaws" four decades later, discussing its cultural impact, box office longevity, and the infamous "Jaws effect" on shark populations.

**19. AGENT**

**Mapping the sections**

I’m organizing sections on Jaws' cultural and industry impact, including its blockbuster model, marketing revolution, societal reactions, and its lasting influence in popular culture and filmmaking.

**20. AGENT**

**Remembering Ebert's words**  
I’m reflecting on Ebert's insight that after 'Jaws', major studios embraced summer seasons for blockbusters, sparking a surge in summer thrillers and FX movies.

**21. AGENT**

**Mapping out key insights**

I'm noting the importance of the lines from the Guinness Book of World Records, which cite "Jaws" as the first summer blockbuster to gross over $100 million in the U.S. box office during its release.

**22. AGENT**

**Mapping the influences**

I'm charting how "Jaws" impacted societal views, consumer habits, blockbuster filmmaking, and pop culture references, examining box office trends, marketing strategies, and the fear of sharks. [1](https://www.bing.com/search?q=sharks%20killed%20after%201975%20trophy%20hunting%20jaws%20effect)

References:
- [1] [Bing Search: 'sharks killed after 1975 trophy hunting jaws effect'](https://www.bing.com/search?q=sharks%20killed%20after%201975%20trophy%20hunting%20jaws%20effect)

**23. AGENT**

**Gathering data**  
Wondering if more sources are needed to cover box office numbers, marketing tactics, societal impacts, and Spielberg's regrets.

**24. AGENT**

**Confirming statistics**

Piecing together these statistics will provide a clear and accurate understanding of the impact of overfishing and the perception of sharks post-Jaws.

**25. AGENT**



**26. AGENT**



**27. AGENT**

**Verifying the claim**

I’m checking Yahoo's snippet, which states Jaws earned over $450 million worldwide. This aligns with the film's extensive global box office.

**28. AGENT**

**Surpassing milestones**

Noticing *Jaws* set records: first $100M in US theaters, $260M domestic, $470M global. Account for 400+ screens and a $700,000 TV ad campaign.

**29. AGENT**

**Crafting the summary**

I’m pulling together key insights on Jaws' impact: its ambitious budget, revolutionary marketing, industry transformation, and pervasive cultural influence are noted. [1](https://www.bing.com/search?q=percentage%20summer%20box%20office%20share%20after%20Jaws%201970s)

References:
- [1] [Bing Search: 'percentage summer box office share after Jaws 1970s'](https://www.bing.com/search?q=percentage%20summer%20box%20office%20share%20after%20Jaws%201970s)

**30. AGENT**

**Tracking cultural influence**

Mapping out Jaws' cultural impact, I noted its tagline, score, public frenzy, and merchandise.

**31. AGENT**



**32. AGENT**

**Evaluating options**

Piecing together methods to embed images directly within HTML. Ideas include using `browser.open` or finding a direct link through `golden.com`suggestions and checking if `impawards` has a direct URL.

**33. AGENT**

**Investigating the cultural impact**

I’m mapping out how 'Jaws' transformed society, shifting beach attendance and consumer behavior. The legacy is evident in pop culture, summer blockbuster strategies, and shark conservation awareness.

**34. AGENT**

**Focusing on credibility**

I’m noting that Daily Jaws might offer credible insights, but I'll prioritize ABC news for beach tourism decline due to their reliable coverage.

**35. AGENT**

**Evaluating image use**

I’m thinking about the Jaws poster for the "Cultural Impact" section, carefully considering guidelines for citing and embedding images.

**36. AGENT**

**Noticing potential issues**  
Wondering if the daily Jaws content isn't visible due to text loading problems. Considering searching for "lines around the block Jaws 1975 photo" as an alternative. [1](https://www.bing.com/search?q=1975%20Jaws%20lines%20block%20photo%20theaters)

References:
- [1] [Bing Search: '1975 Jaws lines block photo theaters'](https://www.bing.com/search?q=1975%20Jaws%20lines%20block%20photo%20theaters)

**37. AGENT**

**Summarizing details**

I’m looking at Remind Magazine snippets, noting the wide theatrical release, impressive opening, and how "Jaws" significantly transformed the summer movie business.

**38. AGENT**

**Tracing influence**

The report details 'Jaws' cultural impact, examining societal attitudes, consumer behavior, movie-making strategies, and its legacy in popular culture, with citations from credible sources.

**39. AGENT**

**Breaking box office records**

Jaws redefined Hollywood with its $100M success and summer blockbuster model.

**40. AGENT**

Final Report:
# The Cultural Impact of *Jaws* (1975) and the Birth of the Summer Blockbuster

Released in 1975, Steven Spielberg’s *Jaws* became a milestone in cinema, redefining both popular culture and Hollywood’s business model. It was an unexpected megahit that shattered box office records and essentially **invented the “summer blockbuster”** as we know it[1](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868),[2](https://www.indiewire.com/features/general/jaws-modern-blockbuster-steven-spielberg-1201844390/). Within months of release, *Jaws* became the first movie in history to earn over $100 million at the U.S. box office[3](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868), eventually grossing around **$260 million domestically** (over $1 **billion** today, adjusted for inflation) and about **$470 million worldwide** during its initial run[4](https://thedailyjaws.com/blog/how-jaws-left-its-indelable-bite-mark-on-pop-culture). This spectacular success – surpassing even *The Godfather* and *The Exorcist* as the highest-grossing film of its time[5](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868) – made *Jaws* a cultural phenomenon. Beyond the numbers, the film’s impact rippled through society: it terrified and thrilled audiences in equal measure, ingraining itself into the public’s imagination and changing how movies were made and marketed thereafter. Below, we examine *Jaws*’ far-reaching cultural impact and its lasting effect on blockbuster filmmaking, with data and examples illustrating its legacy.

## A Record-Breaking Phenomenon and Pop Culture Frenzy

When *Jaws* opened on June 20, 1975, it did so on an **unprecedented 400+ screens nationwide**, a wide release strategy almost unheard of at the time[6](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868). Backed by an intensive marketing campaign – including **$700,000** spent on television advertising (a massive sum in 1975) – Universal Pictures drummed up enormous hype[7](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868). Audiences responded in droves. The film’s release sparked **long lines around theaters (“blockbusters”)**, with many viewers returning for **multiple showings**. These **lines of eager filmgoers**, sometimes wrapping around city blocks, gave rise to the very term *“blockbuster”* in its modern sense. *Jaws* earned an estimated **$7 million in its opening weekend** (a record at the time) and expanded to over 700 screens due to soaring demand. By Labor Day, it had become the **highest-grossing film ever**, overtaking *The Godfather* in just 78 days.

This immediate success was accompanied by *Jaws*-mania in popular culture. The movie’s tagline – *“You’ll never go in the water again”* – proved prophetic, as *Jaws* **literally had people scared to go into the ocean** that summer[8](https://www.usatoday.com/story/entertainment/movies/2025/06/19/jaws-50th-anniversary-movie-summer-blockbuster/83436198007/). The film’s menacing great white shark and its relentless attacks became touchstones for fear, and the catchphrase *“Don’t go in the water”* entered the cultural lexicon. Even those who hadn’t seen the movie were aware of its terrifying reputation. **John Williams’ iconic two-note score** (`dun-DUN...`) became instantly recognizable as a musical embodiment of dread; hearing that simple, ominous theme could make people anxiously glance around for sharks – *even when nowhere near the ocean*. The line *“You’re gonna need a bigger boat,”* ad-libbed by actor Roy Scheider, became one of cinema’s most famous quotes, now a proverbial way to signify being in over one’s head[9](https://www.usatoday.com/story/entertainment/movies/2025/06/19/jaws-50th-anniversary-movie-summer-blockbuster/83436198007/). In these ways, *Jaws* quickly transcended the screen to become a **pop culture touchstone**, referenced in everything from other films and TV shows to everyday conversations.

The *Jaws* phenomenon also turned its young director and cast into stars. The film was a critical success (nominated for the **Best Picture** Oscar and winner of three Academy Awards) and demonstrated that a crowd-pleasing thriller could also earn critical acclaim. This blend of **quality and mass appeal** contributed to the movie’s outsized cultural footprint. As film critic Pauline Kael noted at the time, “It [*Jaws*] is **the most cheerfully perverse scare movie ever made**,” commending its ability to tap into primal fears while entertaining viewers. Moviegoers nationwide shared a communal experience of **screaming and cheering in theaters**, and many left with lasting memories (and new phobias). The impact was so profound that, much as **Hitchcock’s *Psycho* (1960) made people fear showers**, *Jaws* made a whole generation fear what lurked under the waves,. 

## Inventing the Summer Blockbuster Model

Beyond frightening audiences, *Jaws* fundamentally **changed Hollywood’s release strategies** and gave birth to the *modern summer blockbuster* model. Prior to 1975, the summer months were generally considered a **dumping ground for low-budget “B-movies” and exploitation films**, with studios reserving their major releases for winter or awards season[10](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868). *Jaws* turned that wisdom on its head. Seeing the film’s massive summer turnout, studio executives immediately recognized **summer as a prime season for big releases**[11](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868). As legendary critic Roger Ebert wrote, after *Jaws*, **“major Hollywood studios…identified [summer] as the prime releasing season,”** and the film *“inspired hundreds of summer thrillers and [special effects] pictures”* in ensuing years[12](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868). In other words, *Jaws* **ushered in the era of the summer event film**, establishing a tradition wherein each summer’s biggest movie aims to dominate the cultural conversation.

**Marketing and distribution** were key to this new model. Universal’s promotional blitz for *Jaws* was groundbreaking – the studio ultimately spent about **$1.8 million** on marketing (a “whopping” amount at the time) to build up audience anticipation[13](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868). For the first time, a studio leveraged **saturated national TV advertising** in prime time to ensure everyone knew about the film before it opened[14](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868),. The *Jaws* campaign included **30-second TV spots** featuring the terrifying score and killer-shark imagery, months of talk-show appearances for the filmmakers and cast, extensive print ads, and even an updated edition of Peter Benchley’s source novel with the movie poster as its cover,. All of this was designed to create *“pre-release publicity”* at a level never seen before – as *Empire* magazine noted, *Jaws* was *“unequivocally the most influential, important and game-changing summer blockbuster … ushering in new levels of pre-release publicity”*[15](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868). By the time *Jaws* hit theaters, audiences were **already primed with awareness and excitement** for the story and characters, a blueprint that modern blockbusters still follow.

Crucially, *Jaws* was released **widely and simultaneously** – opening on hundreds of screens at once – rather than the old method of rolling out gradually in select cities. This wide-release strategy, combined with heavy marketing, ensured huge opening-weekend grosses and instant nationwide buzz. The results spoke for themselves: *Jaws* **smashed the then-current box office record** and proved that a summer release could deliver monumental returns[16](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868). In the wake of *Jaws*, studios rapidly shifted course. **High-profile action, adventure, and thriller films started claiming summer release dates**, inaugurating what we now call the *summer movie season*. As one observer summarized, *Jaws* **“launched the beginning of the summer blockbuster”** – an industry paradigm where each summer features big-budget spectacles vying for audience attention. Within two years, George Lucas’s *Star Wars* (1977) would follow *Jaws*’ playbook (wide summer release, massive marketing) and break *Jaws*’ box office record, cementing the summer blockbuster model for decades to come[17](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868).

## Changing Hollywood’s Filmmaking Strategies

The success of *Jaws* did not just change *when* studios released movies – it changed *what* movies got made and how. In the late 1970s and beyond, Hollywood increasingly oriented itself around **“tentpole” blockbuster films** with broad audience appeal, a trend sparked by *Jaws*’ triumph[18](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868),[19](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868). Spielberg’s thriller showed that **high-concept, crowd-pleasing storytelling** could yield massive profits. Studios responded by green-lighting more **big-budget spectacle films** in the adventure, sci-fi, and fantasy genres, hoping to replicate *Jaws*’ mass appeal. This marked a shift from the early 1970s era of auteur-driven dramas to a new focus on event entertainment[20](https://www.forbes.com/sites/davidhochman/2025/06/17/jaws-turns-50-the-summer-blockbuster-that-still-dominates-pop-culture/). Shawn Robbins, a box office analyst, notes that genres *“evolved because of what *Jaws* did for summer blockbusters,”* raising the bar for **thrills and special effects** expected by audiences[21](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868). In essence, *Jaws* taught Hollywood that **audiences would flock to theaters for a well-made, thrilling experience**, and studios began chasing those “big thrill” moments (for example, filmmaker J.J. Abrams pointed out that *Star Wars*’ Death Star climax was engineered to give viewers a **similar rush** to *Jaws*’ shark showdown[22](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)).

**Key blockbuster strategies** that emerged in Hollywood after *Jaws* include:

- **Wider Releases and Opening Weekend Focus:** *Jaws* proved the value of opening a film nationwide to maximize impact. Following its lead, studios embraced **saturation booking** (releasing on thousands of screens) and began placing enormous importance on opening-weekend box office tallies as a measure of success,. This “event” release strategy remains standard today for major tentpoles.

- **Mass-Marketing Campaigns:** The film’s innovative use of extensive TV advertising and cross-promotion demonstrated how saturating the media could build hype. Modern blockbusters still rely on **multi-million-dollar marketing blitzes**, from Super Bowl ads to viral campaigns, a practice directly descended from *Jaws*’ playbook[23](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868),.

- **Summer Tentpole Scheduling:** Before *Jaws*, summer was slow at the movies; after *Jaws*, releasing a **marquee film every summer** became de rigueur[24](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868). Studios began staking out prime summer weekends years in advance for their biggest franchise entries, giving birth to the idea of the *“summer popcorn movie”* season that endures today[25](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868),[26](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868).

- **Franchise and Sequel Mentality:** Although sequels were not new, *Jaws* showed their potential in the blockbuster era. Its success led to three sequels in the years after (e.g. *Jaws 2* in 1978) – at a time when multi-film franchises were not yet common. This helped normalize the concept of **serializing hit films into franchises**, a practice that exploded in subsequent decades (e.g. the *Star Wars*, *Indiana Jones*, and *Jurassic Park* series, many spearheaded by Spielberg and Lucas)[27](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868). The modern Hollywood reliance on sequels and cinematic universes can trace roots back to this period.

- **Merchandising and Spin-Off Media:** *Jaws* was among the first films to be treated as a broad *intellectual property (IP)*, not just a standalone movie. Its popularity led to a range of **tie-in merchandise** – from t-shirts and toys to a bestselling soundtrack LP – and even a Universal Studios theme park **“Jaws ride”** in later years[28](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868),. Along with *Star Wars* shortly after, *Jaws* taught Hollywood the profit potential of **ancillary markets** like merchandise, video games, books, and theme park attractions based on a hit film[29](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868). This was a **new revenue model**: a blockbuster could generate income well beyond ticket sales, something every major franchise now aims for.

Perhaps most importantly, *Jaws* revealed that **blockbuster entertainment and quality filmmaking need not be mutually exclusive**. The film earned both massive profits *and* critical praise, disproving the notion that a “popcorn movie” couldn’t also be artful. It was lovingly crafted – with expert editing and an innovative approach to suspense (famously, the malfunctioning mechanical shark forced Spielberg to show less of the creature, *intensifying* the suspense and terror)[30](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html),[31](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html). This taught studios that big-budget genre films, if well-executed, could have broad *and* lasting appeal. *Jaws* has now endured for decades as a classic, whereas many later blockbusters that focused on spectacle over substance have faded away. In the words of one retrospective, *“Jaws showed that movies could make money and appeal broadly and still be of high quality – those things didn’t have to be mutually exclusive.”* This lesson, however, is one that Hollywood has sometimes struggled to heed; the film industry’s pursuit of *Jaws*-sized profits has at times led to formulaic or effects-laden films that lack the character and craftsmanship that made *Jaws* special. Nonetheless, *Jaws*’ influence on filmmaking strategy is undeniable: it **paved the way for today’s tentpole-driven industry**, where summer blockbusters dominate budgets and headlines each year.

## Societal Impact: Fear, Fascination, and “The Jaws Effect”

*Culturally*, *Jaws* had a profound effect on public attitudes – most famously, instilling a deep fear of sharks and the ocean in the collective psyche. **Millions of moviegoers were terrified to go back into the water** after seeing the film’s depiction of a rogue great white shark preying on Amity Island’s beaches[32](https://www.usatoday.com/story/entertainment/movies/2025/06/19/jaws-50th-anniversary-movie-summer-blockbuster/83436198007/). In the **summer of 1975, beach attendance notably dropped** in some areas as people, gripped by fear, chose the movie theater over the seaside. Contemporary reports noted an *“all-out panic”* as swimmers stayed on shore; anecdotal accounts abound of individuals who, decades later, still refused to swim in the ocean because of childhood viewings of *Jaws*. This panic was directly linked to the film’s vivid imagery and masterful suspense – the sense that *something unseen* could attack at any moment tapped into primal human fears. As one commentator quipped, audiences developed a fear of shark-infested waters much like *“they feared showers after *Psycho**,” drawing a parallel between *Jaws*’ impact and an earlier horror milestone.

The **“Jaws effect”** went beyond individual phobias; it significantly shaped public perception of sharks as malevolent, man-eating creatures. Marine experts and historians note that *Jaws* *“transformed the apex predator into an underwater villain whose presence made water unsafe”*, in the public’s mind. The terrifying, almost demonic image of the great white in the film – an implacable giant with black eyes and insatiable bloodlust – left a lasting impression. Suddenly, sharks became the **new public boogeyman** of the deep, replacing whales (which had been depicted as fearsome in tales like *Moby Dick*) as the most feared marine animals. The movie’s *fictional* portrayal of shark behavior (a shark that targets humans and “hunts” people out of spite) was of course biologically inaccurate, but many viewers didn’t separate fact from Hollywood fiction. As one shark researcher observed, *“Jaws villainized sharks, and people became absolutely terrified of any species that was in the ocean”*.

This surge in fear had **real-world consequences for shark populations and marine ecosystems**. In the years after *Jaws*’ release, there was a documented boom in **shark hunting and fishing tournaments**, as some fishermen sought trophies and the public largely approved of killing sharks on sight[33](https://www.smithsonianmag.com/smart-news/steven-spielberg-regrets-how-jaws-impacted-real-world-sharks-180981335/),[34](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html). Scientists point out a *“direct correlation”* between the film’s popularity and declines in shark populations due to sport fishing and culling activities[35](https://www.smithsonianmag.com/smart-news/steven-spielberg-regrets-how-jaws-impacted-real-world-sharks-180981335/). For example, along the U.S. East Coast, the number of large sharks (like great whites, tiger sharks, etc.) was estimated to have **dropped by about 50% in the decade following 1975**, partly attributed to an increase in shark hunting spurred by hysteria from the movie[36](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html). As one expert put it, *“because people’s perceptions of sharks were negative, it made it easier for them to allow and justify overfishing of sharks”* across many species[37](https://www.smithsonianmag.com/smart-news/steven-spielberg-regrets-how-jaws-impacted-real-world-sharks-180981335/). This phenomenon has been termed the **“Jaws effect”** by marine policy scholars – an unwitting legacy of the film is that it seeded such fear and loathing of sharks that it impeded conservation efforts for years[38](https://www.smithsonianmag.com/smart-news/steven-spielberg-regrets-how-jaws-impacted-real-world-sharks-180981335/).

Over time, both Peter Benchley (the author of *Jaws*) and Steven Spielberg expressed regret about this aspect of the movie’s impact[39](https://www.smithsonianmag.com/smart-news/steven-spielberg-regrets-how-jaws-impacted-real-world-sharks-180981335/),[40](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html). Spielberg has said he *“truly, to this day, regrets the decimation of the shark population because of the book and the film”*[41](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html). He mused that he sometimes fears sharks might hold a grudge against him for the worldwide *“feeding frenzy of crazy sport fishermen”* that followed *Jaws*’ release[42](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html). Benchley, for his part, became an active ocean conservationist later in life, openly apologizing for portraying sharks as villains. *“Sharks don’t target human beings, and they certainly don’t hold grudges,”* Benchley said in one interview, emphasizing that the monster shark of *Jaws* was pure fiction and that real sharks rarely attack humans unprovoked[43](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html). These admissions underscore how significant the film’s cultural impact was – its **mythology overpowered scientific reality** in the public mind.

Fortunately, the decades since *Jaws* have seen some shift from fear to **fascination**. Public education, nature documentaries, and initiatives like Discovery Channel’s *Shark Week* (launched in 1988) have helped demystify sharks and highlight their ecological importance. As one marine biologist noted, in recent years *“the fascination [with sharks] has outlasted and outpaced the fear”*[44](https://www.smithsonianmag.com/smart-news/steven-spielberg-regrets-how-jaws-impacted-real-world-sharks-180981335/). Shark tourism has also become popular (e.g. cage-diving experiences), suggesting that many people now seek to appreciate sharks up close rather than kill them,. In fact, a 2020s study cited by *Forbes* found that **Florida’s shark tourism industry** alone brings in over **$221 million a year** – a reminder that sharks can be more valuable alive than dead. This evolving attitude can be seen as a long-term cultural correction to the initial *Jaws*-induced panic. Nonetheless, the legacy of *Jaws* in the collective psyche persists: images of giant great whites and the *Jaws* theme music still immediately evoke a **primal shiver**. Even 50 years on, many people admit the movie (or even just its reputation) left them with an instinctual wariness of dark ocean waters at night[45](https://www.indiewire.com/features/general/jaws-modern-blockbuster-steven-spielberg-1201844390/). Spielberg’s thriller tapped into a deep-seated human fear of the unseen dangers below the surface, and that aspect of its cultural impact remains potent.

## Lasting Legacy in Popular Culture and Blockbuster Filmmaking

Five decades after its release, *Jaws* remains a **towering icon of popular culture** – and the blueprint for the modern blockbuster. Its influence is evident in everything from filmmaking trends to everyday pop references. Each summer, as new big-budget movies hit theaters, industry watchers draw comparisons to *Jaws*, asking whether any film can recreate its mix of critical acclaim, mass appeal, and cultural resonance. The movie’s **anniversaries** continue to be celebrated with fanfare: for its 50th anniversary in 2025, Universal and others rolled out special screenings, a new documentary (*Jaws @ 50*), themed merchandise lines, and even *Jaws*-themed events at baseball games,. Such celebrations underscore how deeply *Jaws* is ingrained in the cultural fabric – it’s not just a film, but a piece of Americana.

In Hollywood, the **“summer blockbuster template”** that *Jaws* created is still the dominant business model. Every year, studios release their tentpole franchise installments in the summer months, often accompanied by massive marketing campaigns and merchandise tie-ins – all in the hopes of spawning the *next* *Jaws* or *Star Wars*. The “epicenter of culture” that *Jaws* established around the movie theater in summer continues: even in an era of streaming, a true blockbuster film (e.g. a superhero epic or a long-awaited sequel) is seen as a *communal summer experience*[46](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868). Paul Dergarabedian, a senior media analyst, noted that the *“sensation ‘Jaws’ created made the movie theater experience the epicenter of culture”*, turning going to the movies into a must-do summer event[47](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868). This pattern endures – recent examples like the record-breaking success of *Barbie* (2023) or *Avengers: Endgame* (2019) on summer release illustrate that the phenomenon *Jaws* started is alive and well.

*Culturally*, *Jaws*’ legacy manifests in countless ways. The film’s **iconography** – the giant shark, the simple poster of a swimmer above an open-mouthed shark, the ominous theme music – is recognized worldwide. The *Jaws* poster (with the lurking great white beneath an unsuspecting swimmer) is considered one of the most famous movie posters ever, often imitated and parodied in other art[48](https://www.usatoday.com/story/entertainment/movies/2025/06/19/jaws-50th-anniversary-movie-summer-blockbuster/83436198007/). The two-note musical motif is a staple shorthand for danger or suspense, used playfully in contexts far removed from sharks (you’ll hear it hummed on beaches, played at sports events, and referenced in other scores whenever a “predator” is near). The movie’s quotable lines (*“Smile, you son of a—!”*, *“We’re gonna need a bigger boat”*, etc.) have entered the vernacular and are still referenced in media and everyday jokes[49](https://www.usatoday.com/story/entertainment/movies/2025/06/19/jaws-50th-anniversary-movie-summer-blockbuster/83436198007/). For example, saying *“bigger boat”* in a business meeting or news headline is a wink that most people understand, even if they haven’t seen the film itself.

The story of *Jaws* – a man versus nature thriller with a relatable everyman hero – also became a narrative template reused in various forms. Countless subsequent movies tried to capture its formula of an ordinary community terrorized by an extraordinary menace, be it other animal-attack films (*Orca*, *Piranha*, *Deep Blue Sea*, *The Meg*) or even non-animal threats. The term *“Jaws in space”* was famously used to pitch *Alien* (1979) during its development, showing how the concept of *Jaws* had become a shorthand in Hollywood for a gripping, high-concept thriller[50](https://www.forbes.com/sites/davidhochman/2025/06/17/jaws-turns-50-the-summer-blockbuster-that-still-dominates-pop-culture/). In the film world, homages to *Jaws* are plentiful: directors often cite it as inspiration, and scenes referencing *Jaws*’ style (the *underwater POV shots*, the sudden scares, the use of music cues) appear in horror and action films across the decades . Entire documentaries and books have been devoted to the making of *Jaws* and its legacy, reinforcing its status as a turning point in movie history.

On the **consumer behavior** side, *Jaws* demonstrated the moneymaking power of broad-appeal entertainment, changing how audiences consume movies. It normalized the idea of the *“summer popcorn movie”*: a fun, thrilling film that you *have* to see with a crowd, and perhaps see more than once. The film also kickstarted the concept of **movies as multi-platform brands**. As noted, after *Jaws*, it became routine for big movies to have spin-off products. Today, the scale of movie merchandising has exploded – something like a Marvel or Disney film launches with tie-in toys, clothing, video games, fast-food promotions, and more. *Jaws*’ own merchandising in the 1970s (though modest by current standards) included everything from shark plush toys to board games, and its continued popularity supports a market for collectibles (vintage *Jaws* posters, models of the shark, etc.) . Even niche items like *Jaws*-themed **wine and beach apparel** have been created for devoted fans in recent years,. The fact that people are willing to buy a $23 “Amity Island-scented” candle or dress their toddlers in *Jaws* T-shirts (as mentioned in a 2025 Forbes piece) speaks to the movie’s enduring *brand* power,. In essence, *Jaws* taught consumers to love the *event* of a blockbuster – not just the film but the whole experience around it.

Finally, *Jaws*’ enduring impact can be summarized by the way it **reshaped Hollywood’s revenue model**. As one industry analyst reflected, *“*Jaws* was the perfect movie at the right time to become an absolute sensation and in turn **changed the whole model on which Hollywood based its revenue-generating capabilities**. Nothing was ever the same after *Jaws*.”*[51](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868). In concrete terms, the film’s success led to bigger budgets (studios realized spending more could yield huge returns – *Jaws* itself cost $12 million, then high for a film, but it earned back far more[52](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)), front-loaded distribution, and the prioritization of blockbuster tentpoles in studio slates. Even the concept of the modern blockbuster “season” – with movies strategically released to maximize audience turnout – is a legacy of *Jaws*. Each year’s summer movie lineup, replete with its spectacle and hype, is a tribute to that pioneering 1975 thriller about a great white shark. And culturally, whenever people hear those shark-like cello notes or eye the ocean’s surface a little nervously, *Jaws*’ shadow looms large.

**In conclusion**, *Jaws* (1975) left an indelible mark both on society and on cinema. It scared people out of the water and into theaters, demonstrating the power of movies to shape attitudes and behaviors. At the same time, it revolutionized the film industry’s approach to big releases – giving birth to the summer blockbuster model that still dominates today’s box office strategy. With measurable milestones (first $100M grosser, over $470M worldwide[53](https://thedailyjaws.com/blog/how-jaws-left-its-indelable-bite-mark-on-pop-culture), decades as one of the top-grossing films of all time) and tangible effects (from **drops in beach tourism** to **booms in merchandise and sequels**[54](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868),), *Jaws* stands as a case study in cultural impact. Its legacy lives on every summer and in every nerve-jangling note of its theme music – a testament to how a single film can change the entertainment landscape and become a permanent part of popular culture. As the Guinness Book of World Records noted about this trailblazing movie, *“people [queued] up around the block to see [it] … [it was] the first film to reach more than $100 million in U.S. box office receipts”*[55](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868) – and in doing so, *Jaws* ensured that neither Hollywood nor audiences would ever look at summer (or the ocean) the same way again. 

**Sources:**

- USA Today – *“Jaws’ 50th anniversary: Movie’s box office impact lives on”*[56](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868),[57](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868),[58](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)  
- ABC News – *“‘Jaws’ Launched Summer Blockbuster 35 Years Ago”*,; *“50th anniversary of 'Jaws': How the film impacted public perception of sharks”*,[59](https://www.smithsonianmag.com/smart-news/steven-spielberg-regrets-how-jaws-impacted-real-world-sharks-180981335/)  
- IndieWire – *“How ‘Jaws’ Forever Changed the Modern Day Blockbuster”*[60](https://thedailyjaws.com/blog/how-jaws-left-its-indelable-bite-mark-on-pop-culture),  
- Yahoo News/News-Herald – *“‘Jaws’ at 50: The summer blockbuster that changed movies forever”*[61](https://www.usatoday.com/story/entertainment/movies/2025/06/19/jaws-50th-anniversary-movie-summer-blockbuster/83436198007/),[62](https://www.usatoday.com/story/entertainment/movies/2025/06/19/jaws-50th-anniversary-movie-summer-blockbuster/83436198007/)  
- *Remind Magazine* – *“Jaws… We Are Still Afraid of the Water”*,  
- Forbes – *“‘Jaws’ Turns 50: The Summer Blockbuster That Still Dominates Pop Culture”*,  
- Smithsonian Magazine – *“Steven Spielberg Regrets How ‘Jaws’ Impacted Real-World Sharks”*[63](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html),[64](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html)

References:
- [1] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [2] [How 'Jaws' Forever Changed the Modern Day Blockbuster - IndieWire](https://www.indiewire.com/features/general/jaws-modern-blockbuster-steven-spielberg-1201844390/)
- [3] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [4] [How JAWS left it's indelable bite-mark on pop culture](https://thedailyjaws.com/blog/how-jaws-left-its-indelable-bite-mark-on-pop-culture)
- [5] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [6] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [7] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [8] ['Jaws' 50th anniversary: Movie's box office impact lives on - USA TODAY](https://www.usatoday.com/story/entertainment/movies/2025/06/19/jaws-50th-anniversary-movie-summer-blockbuster/83436198007/)
- [9] ['Jaws' 50th anniversary: Movie's box office impact lives on - USA TODAY](https://www.usatoday.com/story/entertainment/movies/2025/06/19/jaws-50th-anniversary-movie-summer-blockbuster/83436198007/)
- [10] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [11] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [12] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [13] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [14] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [15] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [16] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [17] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [18] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [19] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [20] [‘Jaws’ Turns 50: The Summer Blockbuster That Still Dominates Pop Culture](https://www.forbes.com/sites/davidhochman/2025/06/17/jaws-turns-50-the-summer-blockbuster-that-still-dominates-pop-culture/)
- [21] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [22] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [23] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [24] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [25] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [26] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [27] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [28] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [29] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [30] ['Jaws' at 50: The summer blockbuster that changed movies forever - Yahoo](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html)
- [31] ['Jaws' at 50: The summer blockbuster that changed movies forever - Yahoo](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html)
- [32] ['Jaws' 50th anniversary: Movie's box office impact lives on - USA TODAY](https://www.usatoday.com/story/entertainment/movies/2025/06/19/jaws-50th-anniversary-movie-summer-blockbuster/83436198007/)
- [33] [Steven Spielberg Regrets How ‘Jaws’ Impacted Real-World Sharks](https://www.smithsonianmag.com/smart-news/steven-spielberg-regrets-how-jaws-impacted-real-world-sharks-180981335/)
- [34] ['Jaws' at 50: The summer blockbuster that changed movies forever - Yahoo](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html)
- [35] [Steven Spielberg Regrets How ‘Jaws’ Impacted Real-World Sharks](https://www.smithsonianmag.com/smart-news/steven-spielberg-regrets-how-jaws-impacted-real-world-sharks-180981335/)
- [36] ['Jaws' at 50: The summer blockbuster that changed movies forever - Yahoo](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html)
- [37] [Steven Spielberg Regrets How ‘Jaws’ Impacted Real-World Sharks](https://www.smithsonianmag.com/smart-news/steven-spielberg-regrets-how-jaws-impacted-real-world-sharks-180981335/)
- [38] [Steven Spielberg Regrets How ‘Jaws’ Impacted Real-World Sharks](https://www.smithsonianmag.com/smart-news/steven-spielberg-regrets-how-jaws-impacted-real-world-sharks-180981335/)
- [39] [Steven Spielberg Regrets How ‘Jaws’ Impacted Real-World Sharks](https://www.smithsonianmag.com/smart-news/steven-spielberg-regrets-how-jaws-impacted-real-world-sharks-180981335/)
- [40] ['Jaws' at 50: The summer blockbuster that changed movies forever - Yahoo](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html)
- [41] ['Jaws' at 50: The summer blockbuster that changed movies forever - Yahoo](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html)
- [42] ['Jaws' at 50: The summer blockbuster that changed movies forever - Yahoo](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html)
- [43] ['Jaws' at 50: The summer blockbuster that changed movies forever - Yahoo](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html)
- [44] [Steven Spielberg Regrets How ‘Jaws’ Impacted Real-World Sharks](https://www.smithsonianmag.com/smart-news/steven-spielberg-regrets-how-jaws-impacted-real-world-sharks-180981335/)
- [45] [How 'Jaws' Forever Changed the Modern Day Blockbuster - IndieWire](https://www.indiewire.com/features/general/jaws-modern-blockbuster-steven-spielberg-1201844390/)
- [46] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [47] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [48] ['Jaws' 50th anniversary: Movie's box office impact lives on - USA TODAY](https://www.usatoday.com/story/entertainment/movies/2025/06/19/jaws-50th-anniversary-movie-summer-blockbuster/83436198007/)
- [49] ['Jaws' 50th anniversary: Movie's box office impact lives on - USA TODAY](https://www.usatoday.com/story/entertainment/movies/2025/06/19/jaws-50th-anniversary-movie-summer-blockbuster/83436198007/)
- [50] [‘Jaws’ Turns 50: The Summer Blockbuster That Still Dominates Pop Culture](https://www.forbes.com/sites/davidhochman/2025/06/17/jaws-turns-50-the-summer-blockbuster-that-still-dominates-pop-culture/)
- [51] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [52] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [53] [How JAWS left it's indelable bite-mark on pop culture](https://thedailyjaws.com/blog/how-jaws-left-its-indelable-bite-mark-on-pop-culture)
- [54] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [55] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [56] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [57] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [58] ['Jaws' Launched Summer Blockbuster 35 Years Ago - ABC News](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868)
- [59] [Steven Spielberg Regrets How ‘Jaws’ Impacted Real-World Sharks](https://www.smithsonianmag.com/smart-news/steven-spielberg-regrets-how-jaws-impacted-real-world-sharks-180981335/)
- [60] [How JAWS left it's indelable bite-mark on pop culture](https://thedailyjaws.com/blog/how-jaws-left-its-indelable-bite-mark-on-pop-culture)
- [61] ['Jaws' 50th anniversary: Movie's box office impact lives on - USA TODAY](https://www.usatoday.com/story/entertainment/movies/2025/06/19/jaws-50th-anniversary-movie-summer-blockbuster/83436198007/)
- [62] ['Jaws' 50th anniversary: Movie's box office impact lives on - USA TODAY](https://www.usatoday.com/story/entertainment/movies/2025/06/19/jaws-50th-anniversary-movie-summer-blockbuster/83436198007/)
- [63] ['Jaws' at 50: The summer blockbuster that changed movies forever - Yahoo](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html)
- [64] ['Jaws' at 50: The summer blockbuster that changed movies forever - Yahoo](https://www.yahoo.com/entertainment/articles/jaws-50-summer-blockbuster-changed-035900989.html)


{'thread_id': 'thread_aEiEPYq61wtSiZcmBR6FLWLX',
 'status': 'completed',
 'progress': 'Azure status: RunStatus.COMPLETED',
 'messages': [{'role': 'user',
   'content': 'Write a report on the cultural impact of the movie Jaws and the effect it had on blockbuster filmmaking',
   'references': [],
   'created_at': '2025-09-08T21:48:41+00:00'},
  {'role': 'agent',
   'content': "I will analyze the cultural impact of the movie 'Jaws' and its effects on blockbuster filmmaking. Expect insights on societal influences, industry transformation, and detailed references. I'll provide a well-organized report shortly.\n\nTitle: Cultural Impact of Jaws and Its Effect on Blockbuster Filmmaking\n\n Starting deep research...",
   'references': [],
   'created_at': '2025-09-08T21:48:46+00:00'},
  {'role': 'agent',
   'content': "**Examining cultural shifts**\n\nI'm digging into how 'Jaws' (1975) impacted societal attitudes, consumer behavior, and movie-making strategies. This involves analyzing box offic

## 7. Inspect Messages & References
Display the messages and references returned when the job completes. Agent messages may contain inline citation formats.

In [9]:
# 7. Inspect messages & references
from pprint import pprint

if status_payload["messages"]:
    for m in status_payload["messages"]:
        print("\n---", m["role"].upper(), "---")
        print(m["content"])
        if m.get("references"):
            print("References:")
            for r in m["references"]:
                print(f"  [{r['citation_number']}] {r['title']} - {r['url']}")
else:
    print("No messages returned.")


--- USER ---
Write a report on the cultural impact of the movie Jaws and the effect it had on blockbuster filmmaking

--- AGENT ---
I will analyze the cultural impact of the movie 'Jaws' and its effects on blockbuster filmmaking. Expect insights on societal influences, industry transformation, and detailed references. I'll provide a well-organized report shortly.

Title: Cultural Impact of Jaws and Its Effect on Blockbuster Filmmaking

 Starting deep research...

--- AGENT ---
**Examining cultural shifts**

I'm digging into how 'Jaws' (1975) impacted societal attitudes, consumer behavior, and movie-making strategies. This involves analyzing box office trends and its lasting cultural legacy. [1](https://www.bing.com/search?q=Jaws%201975%20cultural%20impact%20summer%20blockbuster)
References:
  [1] Bing Search: 'Jaws 1975 cultural impact summer blockbuster' - https://www.bing.com/search?q=Jaws%201975%20cultural%20impact%20summer%20blockbuster

--- AGENT ---
**Evaluating sources**

I’m e

## 8. Download Research Report
If the job completed successfully, download the Markdown and optionally the PDF version of the report from `/deep_research/{thread_id}/{run_id}/download`.

In [10]:
# 8. Download report (Markdown + PDF)
if status_payload["status"].lower() == "completed":
    md_resp = requests.get(f"{BASE_URL}/deep_research/{thread_id}/{run_id}/download", params={"format": "md"})
    md_resp.raise_for_status()
    md_content = md_resp.json()["content"]
    print("Markdown report preview (first 500 chars):\n")
    print(md_content[:500])

    pdf_resp = requests.get(f"{BASE_URL}/deep_research/{thread_id}/{run_id}/download", params={"format": "pdf"})
    pdf_resp.raise_for_status()
    pdf_b64 = pdf_resp.json()["content"]
    print("\nPDF (base64) length:", len(pdf_b64))
else:
    print("Run did not complete successfully; skipping report download.")

Markdown report preview (first 500 chars):

Final Report:
# The Cultural Impact of *Jaws* (1975) and the Birth of the Summer Blockbuster

Released in 1975, Steven Spielberg’s *Jaws* became a milestone in cinema, redefining both popular culture and Hollywood’s business model. It was an unexpected megahit that shattered box office records and essentially **invented the “summer blockbuster”** as we know it [[1]](https://abcnews.go.com/Entertainment/jaws-launched-summer-blockbuster-35-years-ago/story?id=10855868), [[2]](https://www.indiewire.

PDF (base64) length: 84712

PDF (base64) length: 84712
